In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import seaborn as sns
import time

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer= WordNetLemmatizer()
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer

import warnings
warnings.filterwarnings('ignore')

# Enable logging
import logging
logging.basicConfig(level= logging.INFO)

# You will find more libraries as they come in use!


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df_nltk = pd.read_csv('nltk.csv')

In [ ]:
df = pd.read_csv('all_data_FullDate_someLackrating_someLackPrice.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35241 entries, 0 to 35240
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Review Date  35240 non-null  object 
 1   rating       5114 non-null   float64
 2   Product      21581 non-null  object 
 3   Review       35240 non-null  object 
 4   country      32118 non-null  object 
 5   Price        19147 non-null  float64
dtypes: float64(2), object(4)
memory usage: 1.6+ MB


In [ ]:
df_nltk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35241 entries, 0 to 35240
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   35241 non-null  int64  
 1   Review Date  35240 non-null  object 
 2   rating       5114 non-null   float64
 3   Product      21581 non-null  object 
 4   Review       35191 non-null  object 
 5   country      32118 non-null  object 
 6   Price        19147 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 1.9+ MB


In [ ]:
df['pre_wordnet'] = df_nltk['Review']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35241 entries, 0 to 35240
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Review Date  35240 non-null  object 
 1   rating       5114 non-null   float64
 2   Product      21581 non-null  object 
 3   Review       35240 non-null  object 
 4   country      32118 non-null  object 
 5   Price        19147 non-null  float64
 6   pre_wordnet  35191 non-null  object 
dtypes: float64(2), object(5)
memory usage: 1.9+ MB


In [ ]:
df['Review Date'] = pd.to_datetime(df['Review Date'])
df['Review'] = df['Review'].astype(str)

In [ ]:
# Review word count

df['rev_word_count']= df['Review'].apply(lambda x: len(x.strip().split()))

# Unique word count
df['unique_word_count']= df['Review'].apply(lambda x: len(set(str(x).split())))

In [ ]:
df.iloc[0:1]

,Review Date,rating,Product,Review,country,Price,pre_wordnet,rev_word_count,unique_word_count
0,2006-03-25,NaN,B0195AKAEG,Big Farce These chairs lack one major adjustme...,United States,694.0,"big farce lack major adjustment , ability move...",281,174


In [ ]:
df['rev_word_count'].describe()

count    35241.000000
mean        58.004597
std         86.349157
min          1.000000
25%         13.000000
50%         31.000000
75%         68.000000
max       3003.000000
Name: rev_word_count, dtype: float64

In [ ]:
review_min=[]
for val, cnt in df.rev_word_count.value_counts().iteritems():
  if(cnt<10):
        review_min.append(val)
print("Review < 10 words: ",len(review_min))

Review < 10 words:  375


Roberta Pretrained Model


Use a model trained of a large corpus of data

( fine-tuned version of nlptown/bert-base-multilingual-uncased-sentiment on an Amazon US Customer Reviews Dataset. )


Transformer model accounts for the words but also the context related to other words.

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("LiYuan/amazon-review-sentiment-analysis")

model = AutoModelForSequenceClassification.from_pretrained("LiYuan/amazon-review-sentiment-analysis")


tokenizer_config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/670M [00:00<?, ?B/s]

In [ ]:
import torch
example = 'Big Farce These chairs lack one major adjustment, the ability to move the seat out, away from the back. While the major focus is on your back, they forgot to consider your legs.'

# Tokenize the text
inputs = tokenizer(example, padding=True, truncation=True, return_tensors="pt")

# Predict sentiment
with torch.no_grad():
    outputs = model(**inputs)

# Apply softmax to logits
probabilities = torch.nn.functional.softmax(outputs.logits, dim=1).numpy().flatten()

# Output the probabilities
print(probabilities)
print(outputs)

[0.23037155 0.45438254 0.2783992  0.03036081 0.00648583]
SequenceClassifierOutput(loss=None, logits=tensor([[ 1.0583,  1.7376,  1.2477, -0.9682, -2.5117]]), hidden_states=None, attentions=None)


In [ ]:
example2 = "This oatmeal is not good. Its mushy, soft, I don't like it. Quaker Oats is the way to go."
inputs = tokenizer(example2, padding=True, truncation=True, return_tensors="pt")

# Predict sentiment
with torch.no_grad():
    outputs = model(**inputs)

# Apply softmax to logits
probabilities2 = torch.nn.functional.softmax(outputs.logits, dim=1).numpy().flatten()

# Output the probabilities
print(probabilities2)

[0.48211676 0.4250777  0.0824919  0.00712454 0.00318902]


In [ ]:
encoded_text = tokenizer(example, return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    '1' : scores[0],
    '2' : scores[1],
    '3' : scores[2],
    '4' : scores[3],
    '5' : scores[4]
}
print(scores_dict)

{'1': 0.23037156, '2': 0.45438254, '3': 0.2783992, '4': 0.030360814, '5': 0.006485834}


In [ ]:
def get_rate(text):

  encoded_review = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
  output = model(**encoded_review)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  scores_dict = {

      '1' : scores[0],
      '2' : scores[1],
      '3' : scores[2],
      '4' : scores[3],
      '5' : scores[4]
  }
  return scores_dict


In [ ]:
!pip install hdbscan


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 19.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039286 sha256=53f526d7ef2e9b15e7522977bf8ec9b9d2c648c27f40af0112defcd22646797d
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
Successfully built hdbscan
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.6
    Uninstalling Cython-3.0.6:
      Successfully uninstalled Cython-3.0.6


In [ ]:
from hdbscan import HDBSCAN
from tqdm import tqdm
import time
tqdm.pandas(desc="Processing")

df['Rate'] = df['Review'].progress_apply(get_rate)


Processing: 100%|██████████| 35241/35241 [26:46<00:00, 21.94it/s]


In [ ]:
df['Rate']

0        {'1': 0.49947384, '2': 0.3739677, '3': 0.09318...
1        {'1': 0.020333217, '2': 0.1326926, '3': 0.5090...
2        {'1': 0.044736285, '2': 0.08876999, '3': 0.235...
3        {'1': 0.027232815, '2': 0.23406953, '3': 0.651...
4        {'1': 0.013349036, '2': 0.30273375, '3': 0.661...
                               ...                        
35236    {'1': 0.6536494, '2': 0.31599236, '3': 0.02829...
35237    {'1': 0.6278926, '2': 0.29816175, '3': 0.06260...
35238    {'1': 0.00087773526, '2': 0.0031675436, '3': 0...
35239    {'1': 0.00563373, '2': 0.036386028, '3': 0.945...
35240    {'1': 0.6153515, '2': 0.34716102, '3': 0.03513...
Name: Rate, Length: 35241, dtype: object

In [ ]:
def get_key_of_max_value(d):
    return max(d, key=d.get)

In [ ]:
df['rating_predict'] = df['Rate'].apply(get_key_of_max_value)

In [ ]:
df.iloc[0:2]

,Review Date,rating,Product,Review,country,Price,pre_wordnet,rev_word_count,unique_word_count,Rate,rating_predict
0,2006-03-25,NaN,B0195AKAEG,Big Farce These chairs lack one major adjustme...,United States,694.0,"big farce lack major adjustment , ability move...",281,174,"{'1': 0.49947384, '2': 0.3739677, '3': 0.09318...",1
1,2007-04-27,NaN,B0195AKAEG,Nice chair but two problems. I have been using...,United States,694.0,nice two problem . using new aeron medium lumb...,191,126,"{'1': 0.020333217, '2': 0.1326926, '3': 0.5090...",3


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
def get_pos_tag(tag):
    """This function is used to get the part-of-speech(POS) for lemmatization"""

    if tag.startswith('N') or tag.startswith('J'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN #default case

In [ ]:
stop_words = stopwords.words("english")

custom_stop_words = [
    'chair', 'ergonomics', 'ergonomic', 'product', 'furniture', 'item', 'https',  'buy', 'purchase', 'order', "for", "account",
    "not","on","FOR", "ACCOUNT","is","This","For","The","the","hi","It","it","and", "And","this","to","It is",'said',
    'say',"one", "like","well", "seat","good","review", "steelcase","Steelcase","out","while","consider","without","enough","would","may",
    "especially","i","get","extremely","unless","please","rather","thing","herman","miller","probably","my","gives","told","order","-","they",
    "if", "them", "although","chairs","non","amazon","con","great",'t3', 'secrete', 'secretelab','ex', '3d','titan', 'evo','hm',"n't",  "'s", "'ve",'wayfair'
]

stop_words.extend(custom_stop_words)

print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
import re
def preprocess(text):
    """ 1. Removes Punctuations
        2. Removes words smaller than 3 letters
        3. Converts into lowercase
        4. Lemmatizes words
        5. Removes Stopwords
    """
    punctuation= list(string.punctuation)
    doc_tokens= nltk.word_tokenize(text)
    word_tokens= [word.lower() for word in doc_tokens if not (word in punctuation or len(word)<=3)]

    # Lemmatize
    pos_tags=nltk.pos_tag(word_tokens)
#     print(pos_tags)
    doc_words=[wordnet_lemmatizer.lemmatize(word, pos=get_pos_tag(tag)) for word, tag in pos_tags]
    doc_words= [word for word in doc_words if word not in stop_words]

    return doc_words



In [ ]:
tqdm.pandas(desc="Processing")
df['word'] = df['Review'].progress_apply(preprocess)

Processing: 100%|██████████| 35241/35241 [01:40<00:00, 349.05it/s]


In [ ]:
# Tried multiple parts of speech and obtained best topic results using Nouns and Adjectives!
def get_nouns_adjs(series):

    " Topic Modeling using only nouns and adjectivesb"

    pos_tags= nltk.pos_tag(series)
    all_adj_nouns= [word for (word, tag) in pos_tags if (tag=="NN" or tag=="NNS" or tag=="JJ")]
    return all_adj_nouns

df["n_adj"] = df['word'].progress_apply(get_nouns_adjs)

Processing: 100%|██████████| 35241/35241 [00:51<00:00, 682.43it/s] 


Step 3: Add bigrams to your corpus using Word2vec model from gensim

In [ ]:
# Importing gensim related libraries
import gensim
from gensim.models.ldamulticore import LdaMulticore
from gensim.corpora import Dictionary
from gensim.models import Phrases
from collections import Counter
from gensim.models import Word2Vec

min_count 参数
含义：min_count参数定义了一个词组需要在整个文档集中出现的最小次数才能被考虑为一个短语。这个参数有助于过滤掉不常见的、偶然出现的词组。

作用：如果设置为10，这意味着任何词组必须在文档集中至少出现10次才会被视为一个有效的短语。这有助于确保识别的短语确实是文档集中常见和重要的词组。

threshold 参数
含义：threshold参数控制着形成短语的严格程度。它是一个用于评估和选择短语的得分阈值，基于统计检验的概念。

作用：这个参数决定了词组成为短语的难易程度。较高的threshold值意味着只有那些非常频繁共同出现，并且超过随机概率出现的词组才会被形成短语。换句话说，只有那些比较“紧密”结合在一起的词组才会被认定为短语。

In [ ]:
docs= list(df['n_adj'])
phrases = gensim.models.Phrases(docs, min_count=10, threshold=20)
bigram_model = gensim.models.phrases.Phraser(phrases)

In [ ]:
def make_bigrams(texts):
    return [bigram_model[doc] for doc in texts]

# Form Bigrams
data_words_bigrams = make_bigrams(docs)

In [ ]:
df['bigram'] = data_words_bigrams

In [ ]:
df.to_csv('full_with_preprocessingdata_rate.csv')

In [ ]:
# Checkout most frequent bigrams :
bigram_counter1= Counter()
for key in phrases.vocab.keys():
    if key not in stopwords.words('english'):
        if len(str(key).split('_'))>1:
            bigram_counter1[key]+=phrases.vocab[key]
bigram_counter1


Counter({'farce_lack': 1,
         'lack_major': 1,
         'major_adjustment': 2,
         'adjustment_ability': 7,
         'ability_major': 1,
         'major_focus': 1,
         'focus_leg': 1,
         'leg_adjustment': 4,
         'adjustment_mid-thigh': 1,
         'mid-thigh_leg': 1,
         'leg_front': 4,
         'front_edge': 56,
         'edge_blood': 1,
         'blood_circulation': 12,
         'circulation_rest': 3,
         'rest_leg': 11,
         'leg_high': 4,
         'high_relief': 1,
         'relief_bend': 1,
         'bend_knee': 7,
         'knee_tuck': 1,
         'tuck_foot': 2,
         'foot_beneath': 1,
         'beneath_prevent': 1,
         'prevent_hard': 2,
         'hard_rock': 50,
         'rock_fool': 2,
         'fool_mesh': 1,
         'mesh_notice': 1,
         'notice_cushion': 6,
         'cushion_hard': 73,
         'rock_sit': 5,
         'sit_comfortable': 71,
         'comfortable_fold': 4,
         'fold_canvas': 1,
         'canvas_out

In [ ]:
from pandas import DataFrame

# Assuming bigram_counter1 is your Counter object
bigram_list = [{'bigram': k, 'count': v} for k, v in bigram_counter1.items()]
df_bigram_gensim = DataFrame(bigram_list)



In [ ]:
# Sort the DataFrame by 'count' in descending order
df_bigram_gensim_sorted = df_bigram_gensim.sort_values(by='count', ascending=False)

# Display the first few rows of the sorted DataFrame
print(df_bigram_gensim_sorted.head())


                bigram  count
70      lumbar_support   3138
3022     easy_assemble   3111
2977         work_home   1776
1430       home_office   1469
187   customer_service   1157


In [ ]:
df_bigram_gensim.to_csv('bigram_by_gensim.csv')

In [ ]:
for key, counts in bigram_counter1.most_common(100):
    print(key,">>>>", counts)



lumbar_support >>>> 3138
easy_assemble >>>> 3111
work_home >>>> 1776
home_office >>>> 1469
customer_service >>>> 1157
comfortable_easy >>>> 998
assemble_comfortable >>>> 597
high_quality >>>> 564
period_time >>>> 478
head_rest >>>> 451
long_period >>>> 449
worth_price >>>> 403
comfortable_office >>>> 403
long_time >>>> 398
easy_comfortable >>>> 387
comfortable_support >>>> 371
price_point >>>> 370
comfortable_adjustable >>>> 358
last_year >>>> 348
office_comfortable >>>> 339
worth_money >>>> 328
comfortable_look >>>> 324
comfortable_work >>>> 319
comfortable_love >>>> 308
super_easy >>>> 292
adjustable_lumbar >>>> 290
support_adjustable >>>> 289
worth_penny >>>> 289
super_comfortable >>>> 283
comfortable_sturdy >>>> 277
comfortable_supportive >>>> 276
adjustable_arm >>>> 272
comfortable_hour >>>> 269
love_comfortable >>>> 267
look_nice >>>> 266
customer_support >>>> 261
sturdy_comfortable >>>> 258
comfortable_long >>>> 254
office_work >>>> 254
look_comfortable >>>> 247
nice_comfortable

In [ ]:
#Feeding the bigrams into a Word2Vec model produces more meaningful bigrams

Contextual Understanding: Word2Vec learns word representations based on the context in which words appear. By treating bigrams as single units, the model captures the specific context in which two words are used together. This leads to a more nuanced understanding of the phrase as a whole.

Semantic Relationships: In natural language, the combination of words can create a meaning that is different from the sum of its parts. For example, "hot dog" as a bigram conveys a different meaning than "hot" and "dog" separately. Feeding bigrams into Word2Vec helps in capturing these semantic relationships.

Reduces Sparsity: When working with individual words, the model might encounter a large number of unique words, many of which occur infrequently. By converting commonly co-occurring words into bigrams, you reduce the vocabulary size and focus on more meaningful and frequently occurring terms.

Improved Phrase Representation: Word2Vec treats each bigram as a single entity, which allows for a more accurate representation of phrases. This is particularly useful in tasks like sentiment analysis, topic modeling, or information retrieval, where understanding the precise meaning of phrases is important.

Handling Ambiguity: Some words have multiple meanings depending on their context. When these words are part of a bigram, the combined phrase often disambiguates the meaning. Word2Vec can capture this clearer meaning when trained on bigrams.

In [ ]:
w2vmodel = Word2Vec(bigram_model[docs], sg=1, hs= 1, seed=33)

bigram_counter = Counter()




In [ ]:
bigram_counter = Counter()

for key in w2vmodel.wv.vocab.keys():
    if key not in stoplist:
        if len(str(key).split("_")) > 1:
            bigram_counter[key] += w2vmodel.wv.vocab[key].count

for key, counts in bigram_counter.most_common(30):
    print(key,">>>>> " ,counts)

In [ ]:
for key, counts in bigram_counter.most_common(30):
    print(key,">>>>> " ,counts)

customer_service >>>>>  1157
head_rest >>>>>  451
long_period >>>>>  449
worth_money >>>>>  328
worth_penny >>>>>  289
lock_place >>>>>  205
super_comfy >>>>>  200
tall_people >>>>>  190
period_time >>>>>  171
front_computer >>>>>  167
roller_blade >>>>>  158
assembly_instruction >>>>>  148
give_star >>>>>  147
memory_foam >>>>>  136
upright_position >>>>>  136
hardwood_floor >>>>>  133
waste_money >>>>>  126
heavy_duty >>>>>  117
long_term >>>>>  115
couple_day >>>>>  114
reasonable_price >>>>>  110
instruction_clear >>>>>  107
foot_touch >>>>>  95
le_minute >>>>>  91
worth_investment >>>>>  87
lock_mechanism >>>>>  85
send_replacement >>>>>  82
return_policy >>>>>  82
allen_wrench >>>>>  80
game_changer >>>>>  78


In [ ]:
# Assuming w2vmodel is your Word2Vec model
w2vmodel.most_similar(positive= ['comfortable'])


AttributeError: ignored

In [ ]:
w2vmodel.wv.most_similar(positive=['invest'])

[('telework', 0.7559179067611694),
 ('chair.after', 0.728977620601654),
 ('splurge', 0.7093944549560547),
 ('investmenti', 0.7085855603218079),
 ('goodbye', 0.7077556848526001),
 ('decided', 0.7069229483604431),
 ('thisi', 0.7062283754348755),
 ('investment', 0.7047380805015564),
 ('investment_health', 0.7029879093170166),
 ('bite_bullet', 0.6944952011108398)]

In [ ]:
w2vmodel.wv.most_similar(positive=["worth_penny","assemble"], negative=["worth_penny"], topn=3)

[('together-', 0.7841789126396179),
 ('assemblethis', 0.7590901255607605),
 ('assembly', 0.7567770481109619)]

In [ ]:
w2vmodel.wv.most_similar(positive=["worth_penny","customer_service"], negative=["worth_penny"], topn=3)

[('communication', 0.7570428848266602),
 ('team', 0.74632728099823),
 ('polite', 0.7432354092597961)]

In [ ]:
w2vmodel.wv.most_similar(positive=["worth_penny","memory_foam"], negative=["worth_penny"], topn=3)

[('squishy', 0.7180060148239136),
 ('plushy', 0.6871163249015808),
 ('softer', 0.6626212000846863)]

Sentiment Analysis

VADER (Valence Aware Dictionary and sentiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media.